In [59]:
# Load Required Packages
import sklearn
import statistics
import scipy as sc
import numpy as np
import pandas as pd
import random
import cvxpy as cp
import matplotlib.pyplot as plt
import multiprocessing as mp
from multiprocessing import Pool
import time
import re
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import mosek
import statsmodels.api as sm
import gc
import json
import pickle
from pmlb import fetch_data
from pmlb import classification_dataset_names, regression_dataset_names
pd.set_option('display.max_columns', None)

# Implementation of Control Burn

In [60]:
def converge_test(sequence, threshold,length):
    diff = np.diff(sequence)
    if len(diff) < (length+1):
        return False
    else:
        return ( max(np.abs(diff[-length:])) < threshold)
    
def build_trees_bag(arg):
    xTrain = arg[0]
    yTrain = arg[1]
    xTest= arg[2]
    yTest= arg[3]
    max_depth= arg[4]
    problem_type = arg[5]
    loss_type = arg[6]
    lambd= arg[7]
    threshold= arg[8] 
    sketch = arg[9]
   
    train = xTrain
    train = train.reset_index().drop('index',axis = 1)
    train['yTrain'] = list(yTrain)
    features = xTrain.columns
    nfeatures = len(features)
    importance_key = pd.DataFrame(features,columns = ['Features'])
    tree_results = []
    i = 0
    depth = 1
    total_trees = 0
    
    for depth in range (1,max_depth+1):
        i = 0
        ### Early Stopping
        early_stop_pred = []
        early_stop_train_err = []
        converged = False
        
        while converged == False:
            train1 = train.sample(n = len(train), replace = True)
        
            yTrain1 = train1['yTrain']
            xTrain1 = train1[features]
            
            if problem_type == 'Regression':
                clf = DecisionTreeRegressor(max_depth = depth)
            elif problem_type == 'Classification':
                clf = DecisionTreeClassifier(max_depth = depth)
        
            clf.fit(xTrain1,yTrain1)
            imp = pd.DataFrame(np.column_stack((xTrain1.columns,clf.feature_importances_)), columns = ['Features','Importances'])
            used = imp[imp['Importances']>0]['Features'].values
            feature_indicator = [int(x in used) for x in features]
            
            if problem_type == 'Regression':
                pred = clf.predict(xTrain[features])
                test_pred = clf.predict(xTest[features])
            elif problem_type == 'Classification':
                if loss_type == 'logistic':
                    pred = clf.predict_proba(xTrain[features])[:,1]
                    test_pred = clf.predict_proba(xTest[features])[:,1]
                elif loss_type == 'hinge':
                    pred = clf.predict(xTrain[features])
                    test_pred = clf.predict(xTest[features])
                
            feature_importances = pd.merge(importance_key,imp, on = 'Features', how = 'left').fillna(0)['Importances'].values
            tree_results.append([pred,feature_indicator,feature_importances, test_pred  ,clf,xTrain1,yTrain1,features])
            i = i+1
            total_trees = total_trees+1
            early_stop_pred.append(pred)
            early_stop_train_err.append(np.sqrt(np.mean((np.mean(early_stop_pred,axis = 0) - yTrain)**2)))
            converged = converge_test(early_stop_train_err,threshold,5)
            
    return tree_results

def solve_step_nonsketch(arg, tree_results):
    xTrain = arg[0]
    yTrain = arg[1]
    xTest= arg[2]
    yTest= arg[3]
    max_depth= arg[4]
    problem_type = arg[5]
    loss_type = arg[6]
    lambd= arg[7]
    threshold= arg[8] 
    optimization_type = arg[9] #penalized or constrained
    
    feature_list = xTrain.columns
    tree_pred = np.transpose(np.array([np.array(row[0]) for row in tree_results]))
    test_pred = np.transpose(np.array([np.array(row[3]) for row in tree_results]))
    indicators = np.transpose(np.array([np.array(row[1]) for row in tree_results]))
    w = cp.Variable(len(tree_results),nonneg=True)
    
    if optimization_type == 'penalized':
        constraints = []
        if problem_type == 'Regression':
            loss = cp.sum_squares(cp.matmul(tree_pred,w)-yTrain) 
            objective = (1/len(yTrain))*loss + lambd*cp.norm(cp.matmul(indicators,w),1)
        elif problem_type == 'Classification':
            if loss_type == 'logistic':
                loss = -cp.sum(cp.multiply(yTrain, tree_pred@ w) - cp.logistic(tree_pred @ w))
                objective = (1/len(yTrain))*loss + lambd*cp.norm(cp.matmul(indicators,w),1)
            elif loss_type == 'hinge':
                loss =  cp.sum(cp.pos(1 - cp.multiply(yTrain, tree_pred @ w)))
                objective =  (1/len(yTrain))*loss + lambd*cp.norm(cp.matmul(indicators,w),1)
  
        
    if optimization_type == 'constrained':
        if problem_type == 'Regression':
            objective = cp.sum_squares(cp.matmul(tree_pred,w)-yTrain) 
        elif problem_type == 'Classification': 
            if loss_type == 'logistic':
                objective = -cp.sum(cp.multiply(yTrain, tree_pred@ w) - cp.logistic(tree_pred @ w))
            elif loss_type == 'hinge':
                objective = cp.sum(cp.pos(1 - cp.multiply(yTrain, tree_pred @ w)))
        constraints = [cp.norm(cp.matmul(indicators,w),1)<= lambd]
    
    prob = cp.Problem(cp.Minimize(objective),constraints)
    prob.solve(solver = cp.MOSEK,mosek_params = {mosek.dparam.optimizer_max_time: 10000.0} )
    weights = np.asarray(w.value)
    low_values_flags = np.abs(weights) < 10**-3  
    weights[low_values_flags] = 0 
    tree_ind = np.where(weights >0)[0]
    
    if len(tree_ind)==0:
        if problem_type == 'Regression':
            return([[],np.sqrt(np.mean((yTest )**2)),0,np.sqrt(np.mean((yTrain )**2))])
        else:
            return([[],.5,0,.5])
    
    importances = np.array([np.array(row[2]) for row in tree_results])
    feature_importances = np.mean(importances[tree_ind],axis = 0)
    nonzero_features = xTrain.columns[np.where(feature_importances >0)[0]]
    
    if problem_type == 'Regression':
        rf = RandomForestRegressor(n_estimators = 100).fit(xTrain[nonzero_features],yTrain)
        test_pred = rf.predict(xTest[nonzero_features])
        train_pred = rf.predict(xTrain[nonzero_features])
        train_error =  np.sqrt(np.mean((yTrain -train_pred)**2))
        test_error = np.sqrt(np.mean((yTest -test_pred)**2))
        return([feature_importances,test_error,len(nonzero_features),train_error])
        
        
    elif problem_type == 'Classification' :
        rf = RandomForestClassifier(n_estimators = 100).fit(xTrain[nonzero_features],yTrain)
        test_pred = rf.predict_proba(xTest[nonzero_features])[:,1]
        train_pred = rf.predict_proba(xTrain[nonzero_features])[:,1]
        train_error = sklearn.metrics.roc_auc_score(yTrain,train_pred)
        test_error = sklearn.metrics.roc_auc_score(yTest,test_pred)
        return([feature_importances,test_error,len(nonzero_features),train_error])
    
def run_experiment(arg,ntrials,features_to_find,search_limit,l_start,method):
    test_error_result = []
    nonzero_result = []
    train_error_result = []
    trial = 0
    while trial < ntrials:
        
        #Build Trees
        tree_results = method(arg)
        LL = 0
        RL = l_start
        to_find = 0
        counter1 = 0
        count_array = []
        while to_find <= features_to_find:
            arg_list = []
            lambd = (LL + RL)/2
               
            arg[7] = lambd
            result = solve_step_nonsketch(arg,tree_results)
            test_acc = result[1]
            nonzero = result[2]
            train_acc = result[3]
            print(nonzero,to_find,lambd)
            #Append Results
            test_error_result.append(test_acc)
            nonzero_result.append(nonzero)
            train_error_result.append(train_acc)
            count_array.append(nonzero)
        
            freq = pd.DataFrame(np.column_stack(np.unique(count_array, return_counts = True)),columns = ['value','counts'])
            count_to_find = freq.loc[freq['value']==to_find]['counts'].values
            
            
            if arg[9] != 'constrained':
                
                if count_to_find > 0 :
                    counter1 = 0
                    RL = lambd
                    LL = 0
                    to_find = to_find + 1

                elif counter1 >= search_limit:
                    counter1 = 0
                    RL = lambd/2
                    LL = 0
                    to_find = to_find + 1


                elif nonzero < to_find:
                    RL = lambd
                    counter1 = counter1 + 1 

                elif nonzero >= to_find:
                    LL = lambd
                    counter1 = counter1 + 1 
                    
            elif arg[9] == 'constrained':
                
                if count_to_find > 0 :
                    counter1 = 0
                    RL = lambd 
                    LL = 0
                    to_find = to_find + 1

                elif counter1 >= search_limit:
                    counter1 = 0
                    RL = l_start
                    LL = 0
                    to_find = to_find + 1
        
                elif nonzero > to_find:
                    RL = lambd
                    counter1 = counter1 + 1 

                elif nonzero <= to_find:
                    LL = lambd
                    counter1 = counter1 + 1 
              
        trial = trial + 1   
    return test_error_result,nonzero_result,train_error_result

def baseline(xTrain,yTrain,xTest,yTest,problem_type,range1):
    
    if problem_type == 'Regression':
        model = RandomForestRegressor(n_estimators = 100)
        base = 1
    if problem_type == 'Classification':
        model = RandomForestClassifier(n_estimators = 100)
        base = 0.5    
    
    rf = model.fit(xTrain,yTrain)
    imp = pd.DataFrame(np.column_stack((xTrain.columns,rf.feature_importances_)),columns = ['features','scores']).sort_values('scores',ascending = False)
    print(imp)
    acc =[]
    n_features = []
    se = []
    for i in range1:

        if i == 0:
            acc.append(base)
            se.append(0)
            n_features.append(i)
            continue
            
        to_use = imp.head(i)['features'].values
        trial = 0
        acc1 = []
        while trial < 1:
            rf1 = model.fit(xTrain[to_use],yTrain)
            
            if problem_type == 'Regression':
                pred = rf1.predict(xTest[to_use])
                acc1.append(np.sqrt(np.mean((yTest-pred)**2)))
                
            if problem_type == 'Classification':
                pred = rf1.predict_proba(xTest[to_use])[:,1]
                acc1.append(sklearn.metrics.roc_auc_score(yTest,pred))
                
            trial = trial+1
            
        acc.append(np.mean(acc1))
        se.append(np.std(acc1))
        n_features.append(i)
    return acc,n_features,se

def plot_tradeoff_curve(test_acc,nonzero,color,label):
    results = pd.DataFrame()
    for i in range(0,len(test_acc)):
        results = results.append(pd.DataFrame(np.column_stack((test_acc[i],nonzero[i])),columns = ['test_acc','nonzero']))
    agg = results.groupby(['nonzero'], as_index=False).agg({'test_acc':['mean','std','count']})
    
    plt.scatter(agg['nonzero'],agg['test_acc']['mean'],color = color,label = label)
    plt.errorbar(agg['nonzero'],agg['test_acc']['mean'], agg['test_acc']['std'],color = color)
    plt.xlabel('Number of Nonzero Features')
    plt.ylabel('Test Error')
    plt.legend()

# Load PMLB Data

In [61]:
dataset_names = ['analcatdata_bankruptcy'
,'analcatdata_boxing2'
,'analcatdata_cyyoung8092'
,'analcatdata_japansolvent'
,'analcatdata_lawsuit'
,'appendicitis'
,'breast_cancer_wisconsin'
,'bupa'
,'diabetes'
,'glass2'
,'haberman'
,'lupus'
,'phoneme'
,'pima'
,'prnn_crabs'
,'prnn_synth'
,'ring'
,'twonorm'
,'wdbc'
,'spectf',
'chess'
,'dis'
,'horse_colic'
,'hypothyroid'
,'colic',
'sonar',
'Hill_Valley_without_noise',
'crx','clean1','tokyo1','spambase','ionosphere','churn',
'Hill_Valley_with_noise','analcatdata_cyyoung9302','australian','biomed',
'buggyCrx','cleve','credit_a','heart_c','heart_h']


dataset = 'chess'

print(dataset)
data = fetch_data(dataset)

y = data['target']
X = data.drop('target',axis = 1)
features = X.columns
X = preprocessing.scale(X)
X = pd.DataFrame(X,columns = features)

chess


# Duplication Step for Semi Synthetic Experiment

In [63]:
to_duplicate = [# add number of times to duplicate]
for col in to_duplicate:
    for i in range(7):
        name_col = col +'dup'+str(i)
        X[name_col] = X[col] + np.random.normal(0,.1,len(X))
importances

,feat,imp
20,A20,0.218904
32,A32,0.182961
9,A09,0.175894
14,A14,0.0418604
34,A34,0.0359373
5,A05,0.0351542
31,A31,0.0324985
6,A06,0.0226181
0,A00,0.0193865
7,A07,0.0192687


# Load Real World Data

In [ ]:
from sklearn.datasets import load_boston
from sklearn.datasets import load_diabetes
from sklearn.datasets import load_breast_cancer

def load_adult():
    data = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
        header=None)
    data.columns = [
            "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
            "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
            "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
        ]
    data['target'] = 0
    data = data.sample(frac = 1)
    data['target'].loc[data['Income']== data['Income'].unique()[1]] = 1
    y = data['target']
    data.drop(['target','Occupation','Income'],axis = 1, inplace = True)
    data = pd.get_dummies(data, columns = ['WorkClass','Education','MaritalStatus','Relationship','Race','Gender'])
    data['NativeCountry'] = data['NativeCountry'] == ' United-States'
    data['NativeCountry'] = data['NativeCountry'].astype(int)
    features = data.columns
    X = preprocessing.scale(data)
    X = pd.DataFrame(X,columns = features)
    #xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3)
    return X,y

def load_audit():
    audit_risk = pd.read_csv("audit_risk.csv")
    trial = pd.read_csv("trial.csv")
    trial.columns = ['Sector_score','LOCATION_ID', 'PARA_A', 'Score_A', 'PARA_B',
           'Score_B',  'TOTAL', 'numbers', 'Marks',
           'Money_Value', 'MONEY_Marks', 'District',
           'Loss', 'LOSS_SCORE', 'History', 'History_score', 'Score', 'Risk_trial' ]
    trial['Score_A'] = trial['Score_A']/10
    trial['Score_B'] = trial['Score_B']/10
    merged_df = pd.merge(audit_risk, trial, how='outer', on = ['History', 'LOCATION_ID', 'Money_Value', 'PARA_A', 'PARA_B',
           'Score', 'Score_A', 'Score_B', 'Sector_score', 'TOTAL', 'numbers'])

    df = merged_df.drop(['Risk_trial'], axis = 1)
    df['Money_Value'] = df['Money_Value'].fillna(df['Money_Value'].median())
    df = df.drop(['Detection_Risk', 'Risk_F'], axis = 1) 
    df = df[(df.LOCATION_ID != 'LOHARU')]
    df = df[(df.LOCATION_ID != 'NUH')]
    df = df[(df.LOCATION_ID != 'SAFIDON')]
    df = df.astype(float)
    df = df.drop_duplicates(keep = 'first')
    df = df.sample(frac=1)
    class_df = df.drop(["Audit_Risk",'Inherent_Risk','Score','TOTAL'], axis = 1)
    y = class_df["Risk"]    
    classification_X = class_df.drop(["Risk"], axis = 1)
    cols = classification_X.columns
    X = preprocessing.scale(classification_X)
    X = pd.DataFrame(X,columns = cols)
    return X,y

# Run Experiment

In [ ]:
max_depth= 10
problem_type = 'Classification'
loss_type = 'logistic'
optimization_type = 'penalized'


lambd=  0.01
threshold= 10**-3
ntrials = 10
features_to_find = min(len(X.columns),10)
search_limit = 20
l_start = 10

bag_test_acc = []
bag_nonzero = []
base_line_acc = []
baseline_nonzero = []
baseline_se = []

# K Fold Experiment
kf = KFold(n_splits=4)
kf.get_n_splits(X)
for train_index, test_index in kf.split(X):
    xTrain, xTest = X.iloc[train_index], X.iloc[test_index]
    yTrain, yTest = y.iloc[train_index], y.iloc[test_index]
    arg = [xTrain,yTrain,xTest,yTest, max_depth,problem_type,loss_type,lambd,threshold,optimization_type]
    bag_test_acc1,bag_nonzero1,bag_train_acc1 = run_experiment(arg,ntrials,features_to_find,search_limit,l_start,build_trees_bag)
    bag_test_acc = np.append(bag_test_acc,bag_test_acc1)
    bag_nonzero = np.append(bag_nonzero,bag_nonzero1)
    
    range1 = np.unique(bag_nonzero1)
    base_line_acc1,baseline_nonzero1,baseline_se1 = baseline(xTrain,yTrain,xTest,yTest,problem_type,range1)
    
    base_line_acc = np.append(base_line_acc,base_line_acc1)
    baseline_nonzero = np.append(baseline_nonzero,baseline_nonzero1)
    
baseline1 = pd.DataFrame(np.column_stack((baseline_nonzero,base_line_acc)),columns = ['nonzero','acc'])
baseline1 = baseline1.groupby('nonzero').agg(['mean','std']).reset_index()
baseline1  

# Plot and Save Results

In [ ]:
plot_tradeoff_curve(bag_test_acc,bag_nonzero,'blue',label = 'ControlBurn')
plt.ylabel('ROC-AUC')
plt.xlabel('Number of Non-Zero Features')
plt.xlim(0,10)
plt.scatter(baseline1['nonzero'],baseline1['acc']['mean'],label = 'Random Forest',color = 'grey')
plt.errorbar(baseline1['nonzero'],baseline1['acc']['mean'], baseline1['acc']['std'],color = 'grey')
plt.title(dataset)
plt.legend()

In [66]:
filename = dataset+'.pkl'
results = {}
results['name'] = dataset
results['baseline_acc'] = base_line_acc
results['baseline_nonzero'] = baseline_nonzero
results['nonzero'] = bag_nonzero
results['acc'] = bag_test_acc
results['nrow'] = len(y)
results['ncol'] = len(X.columns)

with open(filename, 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(filename)

chess373duplicate.pkl
